## overview

In [1]:
# default package
import logging
import sys 
import os
import pathlib
import IPython
import random

In [2]:
# third party package
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import psycopg2
import sqlalchemy
from sqlalchemy.sql.expression import column,select,table
from sqlalchemy import create_engine
import sklearn.datasets

In [3]:
# my package
sys.path.append(os.path.join(pathlib.Path().resolve(),"../"))

In [4]:
# reload settings
%load_ext autoreload
%autoreload 2

In [5]:
# logger
logger=logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [6]:
## grap setting
sns.set()

In [7]:
## db setting
engine = create_engine('postgresql://postgres:postgres@db:5432/postgres')
with engine.connect() as conn:
    print(conn.closed)

False


## sql to python

In [8]:
def ct(file:str,chap_num:int,num:int,sub_num:int=None,engine:sqlalchemy.engine=engine)->None:
    """
    pubulic schema内のtableをすべて削除してから、指定のtable作成
    """
    if sub_num==None: 
        filename=f"../code-ssql2/ddl/ssql2-{file}{chap_num:02}_{num:02}"
    else:
        filename=f"../code-ssql2/ddl/ssql2-{file}{chap_num:02}_{num:02}_{sub_num}"
        
    with open(filename,"r") as f:
        stmt=f.read() 
    with engine.connect() as conn:
        conn.execute("drop schema public cascade; create schema public;")
        conn.execute(stmt)

In [9]:
def exe(stmt:str,engine:sqlalchemy.engine=engine):
    """
    sql実行
    """
    with engine.connect() as conn:
        query=conn.execute(stmt)
        try:
            result=query.fetchall()
            return result
        except:
            return None

## List

In [10]:
ct("List",4,2)
exe("""select distinct 入金額 from 家計簿""")

[(0,), (280000,)]

In [11]:
ct("List",4,2)
exe("""select distinct 費目 from 家計簿""")

[('食費',), ('給料',), ('交際費',), ('教養娯楽費',), ('水道光熱費',)]

In [12]:
ct("List",4,4)
exe("""select * from 家計簿 order by 出金額""")

[(datetime.date(2018, 2, 10), '給料', '1月の給料', 280000, 0),
 (datetime.date(2018, 2, 3), '食費', 'コーヒーを購入', 0, 380),
 (datetime.date(2018, 2, 11), '教養娯楽費', '書籍を購入', 0, 2800),
 (datetime.date(2018, 2, 14), '交際費', '同期会の会費', 0, 5000),
 (datetime.date(2018, 2, 18), '水道光熱費', '1月の電気代', 0, 7560)]

In [13]:
ct("List",4,5)
exe("""select * from 家計簿 order by 日付 desc""")

[(datetime.date(2018, 2, 18), '水道光熱費', '1月の電気代', 0, 7560),
 (datetime.date(2018, 2, 14), '交際費', '同期会の会費', 0, 5000),
 (datetime.date(2018, 2, 11), '教養娯楽費', '書籍を購入', 0, 2800),
 (datetime.date(2018, 2, 10), '給料', '1月の給料', 280000, 0),
 (datetime.date(2018, 2, 3), '食費', 'コーヒーを購入', 0, 380)]

In [14]:
ct("List",4,6)
exe("""select * from 家計簿 
order by 入金額 desc,出金額 desc""")

[(datetime.date(2018, 2, 10), '給料', '1月の給料', 280000, 0),
 (datetime.date(2018, 2, 18), '水道光熱費', '1月の電気代', 0, 7560),
 (datetime.date(2018, 2, 14), '交際費', '同期会の会費', 0, 5000),
 (datetime.date(2018, 2, 11), '教養娯楽費', '書籍を購入', 0, 2800),
 (datetime.date(2018, 2, 3), '食費', 'コーヒーを購入', 0, 380)]

In [15]:
ct("List",4,8)
exe("""
select 費目,出金額 from 家計簿
order by 出金額 desc
offset 0 rows 
fetch next 3 rows only
""")

[('水道光熱費', 7560), ('交際費', 5000), ('教養娯楽費', 2800)]

In [16]:
ct("List",4,9)
exe("""
select 費目,出金額 from 家計簿
order by 出金額 desc
offset 2 rows 
fetch next 1 rows only
""")

[('教養娯楽費', 2800)]

In [17]:
ct("List",4,11)
exe("""
select 費目,入金額,出金額 from 家計簿
union
select 費目,入金額,出金額 from 家計簿アーカイブ
order by 2,3,1
""")

[('食費', 0, 380),
 ('教養娯楽費', 0, 1800),
 ('教養娯楽費', 0, 2800),
 ('水道光熱費', 0, 4200),
 ('交際費', 0, 5000),
 ('食費', 0, 5000),
 ('水道光熱費', 0, 7560),
 ('居住費', 0, 80000),
 ('給料', 280000, 0)]

In [18]:
ct("List",4,12,1)
exe("""select 費目 from 家計簿
except
select 費目 from 家計簿アーカイブ
""")

[('交際費',)]

## Q

In [19]:
ct("Q",4,1,1)
exe("""
select * from 注文履歴
order by 注文番号,注文枝番
""")

[(datetime.date(2018, 1, 1), 101, 1, 'ブレンドコーヒー', '1', 'S', 210, 1, 210),
 (datetime.date(2018, 1, 1), 101, 2, 'サンドイッチA', '2', 'X', 360, 1, 360),
 (datetime.date(2018, 1, 5), 105, 1, 'カフェラテ', '1', 'M', 380, 2, 760)]

In [20]:
ct("Q",4,1,2)
exe("""
select 商品名 from 注文履歴
where 日付>='2018-01-01'
and  日付<='2018-01-31'
order by 商品名
""")

[('カフェラテ',), ('サンドイッチA',), ('ブレンドコーヒー',)]

In [21]:
ct("Q",4,1,3)
exe("""
select 注文番号,注文枝番,注文金額 from 注文履歴
where 分類='1'
order by 注文金額
offset 0 rows
fetch next 3 rows only
""")


[(101, 1, 210), (106, 1, 370), (105, 1, 760)]

In [22]:
ct("Q",4,1,4)
exe("""
select * from 注文履歴
where 分類='3' and 数量>2
order by 日付,数量 desc
""")

[(datetime.date(2018, 1, 3), 103, 1, 'トートバッグ', '3', 'X', 1500, 3, 4500)]

In [23]:
ct("Q",4,2,1)
exe("""
select * from 偶数
union
select * from 奇数
""")

[(10,),
 (2,),
 (-10,),
 (5,),
 (4,),
 (6,),
 (9,),
 (-8,),
 (-7,),
 (-2,),
 (1,),
 (-4,),
 (-5,),
 (-1,),
 (3,),
 (-3,),
 (8,),
 (-9,),
 (-6,),
 (0,),
 (7,)]

In [24]:
ct("Q",4,2,2)
exe("""
select * from 整数
except
select * from 偶数
""")

[(9,), (-5,), (3,), (5,), (-9,), (-1,), (-7,), (7,), (-3,), (1,)]